

# Instalando Darknet



Primero actualizamos o instalamos unas bibliotecas que vamos a utilizar.

In [ ]:
!sudo apt-get install build-essential git libopencv-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
libopencv-dev is already the newest version (4.5.4+dfsg-9ubuntu4).
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


Con este fragmento de código vamos a obtener la arquitectura de la GPU que está utilizando la máquina. Si al final no se encuentra es necesario editar el código con la información en los comentarios para incluir la arquitectura a utilizar ya que se utiliza durante la instalación de darknet.

In [ ]:
!nvidia-smi
# FUENTE: https://colab.research.google.com/drive/1mzL6WyY9BRx4xX476eQdhKDnd_eixBlG
# This cell ensures you have the correct architecture for your respective GPU
# If you command is not found, look through these GPUs, find the respective
# GPU and add them to the archTypes dictionary

# Tesla V100
# ARCH= -gencode arch=compute_70,code=[sm_70,compute_70]

# Tesla K80
# ARCH= -gencode arch=compute_37,code=sm_37

# GeForce RTX 2080 Ti, RTX 2080, RTX 2070, Quadro RTX 8000, Quadro RTX 6000, Quadro RTX 5000, Tesla T4, XNOR Tensor Cores
# ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# Jetson XAVIER
# ARCH= -gencode arch=compute_72,code=[sm_72,compute_72]

# GTX 1080, GTX 1070, GTX 1060, GTX 1050, GTX 1030, Titan Xp, Tesla P40, Tesla P4
# ARCH= -gencode arch=compute_61,code=sm_61

# GP100/Tesla P100 - DGX-1
# ARCH= -gencode arch=compute_60,code=sm_60

# For Jetson TX1, Tegra X1, DRIVE CX, DRIVE PX - uncomment:
# ARCH= -gencode arch=compute_53,code=[sm_53,compute_53]

# For Jetson Tx2 or Drive-PX2 uncomment:
# ARCH= -gencode arch=compute_62,code=[sm_62,compute_62]
import os
os.environ['GPU_TYPE'] = str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read())

def getGPUArch(argument):
  try:
    argument = argument.strip()
    # All Colab GPUs
    archTypes = {
        "Tesla V100-SXM2-16GB": "-gencode arch=compute_70,code=[sm_70,compute_70]",
        "Tesla K80": "-gencode arch=compute_37,code=sm_37",
        "Tesla T4": "-gencode arch=compute_75,code=[sm_75,compute_75]",
        "Tesla P40": "-gencode arch=compute_61,code=sm_61",
        "Tesla P4": "-gencode arch=compute_61,code=sm_61",
        "Tesla P100-PCIE-16GB": "-gencode arch=compute_60,code=sm_60"

      }
    return archTypes[argument]
  except KeyError:
    return "GPU must be added to GPU Commands"
os.environ['ARCH_VALUE'] = getGPUArch(os.environ['GPU_TYPE'])

print("GPU Type: " + os.environ['GPU_TYPE'])
print("ARCH Value: " + os.environ['ARCH_VALUE'])

Sun Nov  5 22:30:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Con este fragmento vamos a instalar darknet. Dentro del parámetro ARCH hay que cambiar a la arquitectura obtenida con el fragmento de código anterior (esto lo hace automaticamente el script).

In [ ]:
%cd /content
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile
!make
!sudo cp libdarknet.so /usr/local/lib/
!sudo cp include/darknet.h /usr/local/include/
!sudo ldconfig
!wget https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights

/content
Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 15833 (delta 157), reused 219 (delta 140), pack-reused 15539
Receiving objects: 100% (15833/15833), 14.72 MiB | 10.03 MiB/s, done.
Resolving deltas: 100% (10591/10591), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set 

# Entrenando

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/darknet
!./darknet detector -dont_show train /content/drive/MyDrive/Training/custom_models/custom_Dataset.data /content/drive/MyDrive/Training/custom_models/custom_yolo4.cfg /content/drive/MyDrive/Training/custom_models/trained_models/custom_yolo4_last.weights

/content/darknet
 CUDA-version: 11080 (12000), cuDNN: 8.9.0, GPU count: 1  
 OpenCV version: 4.5.4
custom_yolo4
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv 

In [ ]:
%cd /content/darknet
!./darknet detector -dont_show train /content/drive/MyDrive/Training/custom_models/custom_Dataset.data /content/drive/MyDrive/Training/custom_models/custom_yolo4.cfg /content/darknet/yolov4.weights -clear

In [ ]:
drive.flush_and_unmount()

In [ ]:
!unzip -u /content/drive/MyDrive/Resources/Dataset_training_and_testing.zip -d /content/drive/MyDrive/Training/custom_models

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005307.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005308.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005309.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005310.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005315.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005316.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005317.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005318.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005319.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005320.txt  
  inflating: /content/drive/MyDrive/Training/custom_models/train/zoom/00005321.txt  
  in